In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import json
#import utm as utm

#*********basic restaurant data
restaurant = pd.read_csv("dataset/restaurants.csv")

# restaurant.dropna(
#     axis=0,
#     how='any',
#     thresh=None,
#     subset=None,
#     inplace=True
# )
restaurant.dropna()

color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(restaurant, 
                        lat="LATITUDE", 
                        lon="LONGITUDE", 
                        hover_name="NAME", 
                        hover_data=["PHYSICALADDRESS", "PHYSICALCITY"],
                        # color="Listed",
                        # color_continuous_scale=color_scale,
                        zoom=9, 
                        height=300)


# fig = go.Figure(go.Scattermapbox())

#lat_lon = utm.to_latlon(508009.1730000004, 5446008.925249999, 10, 'U')
#print(lat_lon)

# fig = px.scatter_mapbox(restaurant, lat="Lat", lon="Long", hover_name="Address", hover_data=["Address", "Listed"],
#                         color_discrete_sequence=["fuchsia"], zoom=3, height=300)


fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
import json
from types import SimpleNamespace
#*********basic transit data
#transit_json = open('dataset/transit.json')

with open('dataset/transit.json') as your_file:
    your_dict = json.load(your_file)
    your_file.seek(0)
    your_root = json.load(your_file, object_hook= lambda x: 
                                        SimpleNamespace(**x))
type(your_root[2])
# transit = json.load(transit_json)
# for i in transit[2]:
#     print(i)

In [ ]:
from pyproj import Transformer, Proj, transform
WGS84 = Proj(init='EPSG:4326')
p = Proj(init='EPSG:26910')
#utm
x,y,z = 502562.2479999997, 5447647.1559999995, 0
lat, lon = transform(p, WGS84, x, y)
print(lat, lon)

In [1]:
from scripts.model import TransitStopRaw
from scripts.data_processing import parse_to_raw_stop_data

transit_stop_array = []

parse_to_raw_stop_data(transit_stop_array)



for i in range(5):
    transitStopEle = transit_stop_array[i]
    print(f"Stop {i+1}:")
    print(f"id: {transitStopEle.id}")
    print(f"x: {transitStopEle.x}")
    print(f"y: {transitStopEle.y}")
    print(f"location: {transitStopEle.location}")
    print("----------------")


Stop 1:
id: 59741
x: 502265.75100000016
y: 5450336.422
location: BURNABY
----------------
Stop 2:
id: 55179
x: 502646.13989999983
y: 5437429.9046
location: DELTA
----------------
Stop 3:
id: 54877
x: 502761.76400000043
y: 5437393.789999999
location: DELTA
----------------
Stop 4:
id: 52367
x: 502562.2479999997
y: 5447647.1559999995
location: RICHMOND
----------------
Stop 5:
id: 52771
x: 503023.61400000006
y: 5451020.790999999
location: BURNABY
----------------
